In [ ]:
from google.oauth2 import service_account
from googleapiclient.http import MediaIoBaseDownload,MediaFileUpload
from googleapiclient.discovery import build
import pprint
import io

# Задаем кол-во отступов
pp = pprint.PrettyPrinter(indent=4)

# -------------- Scopes — это перечень возможностей, которыми будет обладать сервис, созданный в скрипте. ----------------------------------
#  Ниже приведены Scopes, которые относятся к API Google Drive. https://developers.google.com/identity/protocols/oauth2/scopes?hl=ru
# API таблиц Area120 , v1alpha1 
# Объем	Описание
# https://www. googleapis. com/ auth/ drive	Просмотр, редактирование, создание и удаление всех ваших файлов на Google Диске
# https://www. googleapis. com/ auth/ drive. file	Просматривайте, редактируйте, создавайте и удаляйте только те файлы Google Диска, которые вы используете с этим приложением.
# https://www. googleapis. com/ auth/ drive. readonly	Просмотр и загрузка всех ваших файлов на Google Диске
# https://www. googleapis. com/ auth/ spreadsheets	Просмотр, редактирование, создание и удаление всех ваших таблиц Google Sheets
# https://www. googleapis. com/ auth/ spreadsheets. readonly	Просмотреть все ваши таблицы Google Sheets
# https://www. googleapis. com/ auth/ tables	Просмотр, редактирование, создание и удаление таблиц в Tables by Area 120

In [1]:
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials
import pandas as pd

# Авторизация
creds = Credentials.from_service_account_file(
    "creds.json", scopes=["https://www.googleapis.com/auth/spreadsheets.readonly"]
)

service = build("sheets", "v4", credentials=creds)

# ID таблицы и имя вкладки
SPREADSHEET_ID = "1j2H1sGdhuQYsMDs8JkDQ4DEL27pNxXtKVE6JcJbyvzo"
RANGE_NAME = "Настройки автопилота!A1:M596"  # Имя вкладки + диапазон

# Получаем данные
result = service.spreadsheets().values().get(
    spreadsheetId=SPREADSHEET_ID,
    range=RANGE_NAME
).execute()

values = result.get("values", [])

# Конвертируем в DataFrame
if values:
    df = pd.DataFrame(values[1:], columns=values[0])
    df.to_excel("Настройки автопилота.xlsx", index=False)
    print("✅ Лист успешно сохранён в Excel!")
else:
    print("⚠️ Нет данных.")

TimeoutError: The read operation timed out

In [ ]:
import time
import pandas as pd
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials
from google_auth_httplib2 import AuthorizedHttp
import httplib2
from googleapiclient.errors import HttpError

# Авторизация
creds = Credentials.from_service_account_file(
    "creds.json",
    scopes=["https://www.googleapis.com/auth/spreadsheets.readonly"]
)
http = httplib2.Http(timeout=300)
authed_http = AuthorizedHttp(creds, http=http)
service = build("sheets", "v4", http=authed_http)

SPREADSHEET_ID = "1j2H1sGdhuQYsMDs8JkDQ4DEL27pNxXtKVE6JcJbyvzo"
SHEET_NAME = "Настройки автопилота"
STEP = 300  # строк за раз
MAX_ROWS = 2000  # максимум строк, которые планируем прочитать

def get_values_chunk(service, spreadsheet_id, range_name, retries=5):
    """Получает данные с повторными попытками при ошибках 500/503"""
    for attempt in range(1, retries + 1):
        try:
            return service.spreadsheets().values().get(
                spreadsheetId=spreadsheet_id,
                range=range_name
            ).execute().get("values", [])
        except HttpError as e:
            if e.resp.status in [500, 503]:
                wait = attempt * 5
                print(f"⚠️ Ошибка {e.resp.status}, попытка {attempt}/{retries}, ждём {wait} сек...")
                time.sleep(wait)
            else:
                raise
    print("❌ Сервер не ответил после нескольких попыток.")
    return []

def download_sheet_in_parts():
    all_data = []
    start_row = 1
    header = None

    while True:
        end_row = start_row + STEP - 1
        range_name = f"{SHEET_NAME}!A{start_row}:M{end_row}"
        print(f"🔹 Загружаем диапазон: {range_name} ...")

        values = get_values_chunk(service, SPREADSHEET_ID, range_name)
        if not values:
            break

        if not header:
            header = values[0]
            values = values[1:]

        all_data.extend(values)
        if len(values) < STEP - 1:
            break  # значит, дошли до конца
        start_row += STEP

    if all_data:
        df = pd.DataFrame(all_data, columns=header)
        df.to_excel(f"{SHEET_NAME}.xlsx", index=False)
        print(f"✅ Успешно сохранено: {SHEET_NAME}.xlsx ({len(df)} строк) диапазона {range_name}")
    else:
        print("⚠️ Не удалось получить данные.")

download_sheet_in_parts()

🔹 Загружаем диапазон: Настройки автопилота!A1:M300 ...
⚠️ Ошибка 503, попытка 1/5, ждём 5 сек...
⚠️ Ошибка 503, попытка 2/5, ждём 10 сек...
🔹 Загружаем диапазон: Настройки автопилота!A301:M600 ...


ModuleNotFoundError: No module named 'openpyxl'